In [1]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import flopy
import pyemu

In [2]:
m = flopy.modflow.Modflow("freyberg",version="mfnwt",exe_name="mfnwt")

In [7]:
dis = flopy.modflow.ModflowDis(m,nlay=3,nrow=40,ncol=20,nper=2,
                               perlen=[1.0,180.0],top=35,
                               botm=[32.5,30,os.path.join("freyberg_data","botm.ref")],
                               delr=250.0,delc=250.0)

****Warning -- two packages of the same type:  <class 'flopy.modflow.mfdis.ModflowDis'> <class 'flopy.modflow.mfdis.ModflowDis'>
replacing existing Package...


In [8]:
bas = flopy.modflow.ModflowBas(m,ibound=os.path.join("freyberg_data","ibound.ref"),strt=35.)

In [9]:
upw = flopy.modflow.ModflowUpw(m,hk=5.0,ss=0.0001,sy=0.01)

In [10]:
nwt = flopy.modflow.ModflowNwt(m)

In [11]:
oc = flopy.modflow.ModflowOc(m,stress_period_data=None)

In [16]:
wel_i = [9,11,20,26,29,34]
wel_j = [16,13,14,10,6,12]
wel_flux = [-500,-500,-500,-50,-50,-500]
sp0,sp1 = [],[]
for i,j,f in zip(wel_i,wel_j,wel_flux):
    sp0.append([2,i,j,f])
    sp1.append([2,i,j,f*1.25])#more pumping in second stress period
wel = flopy.modflow.ModflowWel(m,stress_period_data={0:sp0,1:sp1})



[[2, 9, 16, -500], [2, 11, 13, -500], [2, 20, 14, -500], [2, 26, 10, -50], [2, 29, 6, -50], [2, 34, 12, -500]]
[[2, 9, 16, -625.0], [2, 11, 13, -625.0], [2, 20, 14, -625.0], [2, 26, 10, -62.5], [2, 29, 6, -62.5], [2, 34, 12, -625.0]]


In [37]:
sfr_j = 15
reach_data = flopy.modflow.ModflowSfr2.get_empty_reach_data(m.nrow)
seg_data = flopy.modflow.ModflowSfr2.get_empty_segment_data(m.nrow)
print(reach_data.dtype)
print(seg_data.dtype)
top_slice = m.dis.top.array[:,sfr_j]
for i in range(m.nrow):
    reach_data[i]["k"] = 0
    reach_data[i]["i"] = i
    reach_data[i]['j'] = sfr_j
    reach_data[i]["iseg"] = i + 1
    reach_data[i]["ireach"] = i + 1
    reach_data[i]["rchlen"] = 250.0
    seg_data[i]["nseg"] = i + 1
    seg_data[i]["icalc"] = 1
    seg_data[i]["outseg"] = i + 2
    seg_data[i]["hcond1"] = 0.1
    seg_data[i]["hcond2"] = 0.1
    seg_data[i]["width1"] = 10.0
    seg_data[i]["width2"] = 10.0
    seg_data[i]["elevup"] = top_slice[i]+0.1
    seg_data[i]["elevdn"] = top_slice[i]-0.1
    seg_data[i]["thickm1"] = 1.0
    seg_data[i]["thickm2"] = 1.0
    seg_data[i]["roughch"] = 0.1
    #seg_data.append(i+1,1,i+2,0,0)
    


(numpy.record, [('node', '<i8'), ('k', '<i8'), ('i', '<i8'), ('j', '<i8'), ('iseg', '<i8'), ('ireach', '<i8'), ('rchlen', '<f4'), ('strtop', '<f4'), ('slope', '<f4'), ('strthick', '<f4'), ('strhc1', '<f4'), ('thts', '<f4'), ('thti', '<f4'), ('eps', '<f4'), ('uhc', '<f4'), ('reachID', '<i8'), ('outreach', '<i8')])
(numpy.record, [('nseg', '<i8'), ('icalc', '<i8'), ('outseg', '<i8'), ('iupseg', '<i8'), ('iprior', '<i8'), ('nstrpts', '<i8'), ('flow', '<f4'), ('runoff', '<f4'), ('etsw', '<f4'), ('pptsw', '<f4'), ('roughch', '<f4'), ('roughbk', '<f4'), ('cdpth', '<f4'), ('fdpth', '<f4'), ('awdth', '<f4'), ('bwdth', '<f4'), ('hcond1', '<f4'), ('thickm1', '<f4'), ('elevup', '<f4'), ('width1', '<f4'), ('depth1', '<f4'), ('thts1', '<f4'), ('thti1', '<f4'), ('eps1', '<f4'), ('uhc1', '<f4'), ('hcond2', '<f4'), ('thickm2', '<f4'), ('elevdn', '<f4'), ('width2', '<f4'), ('depth2', '<f4'), ('thts2', '<f4'), ('thti2', '<f4'), ('eps2', '<f4'), ('uhc2', '<f4')])


In [38]:
sfr = flopy.modflow.ModflowSfr2(m,reach_data=reach_data,segment_data=seg_data)

****Warning -- two packages of the same type:  <class 'flopy.modflow.mfsfr2.ModflowSfr2'> <class 'flopy.modflow.mfsfr2.ModflowSfr2'>
replacing existing Package...


/Users/jeremyw/miniconda3/lib/python3.6/site-packages/numpy/core/records.py:507: FutureWarning: Numpy has detected that you may be viewing or writing to an array returned by selecting multiple fields in a structured array. 

This code may break in numpy 1.13 because this will return a view instead of a copy -- see release notes for details.
  return obj.view(dtype=(self.dtype.type, obj.dtype))


In [39]:
m.model_ws = "freyberg_setup"
m.write_input()


changing model workspace...
   freyberg_setup


/Users/jeremyw/miniconda3/lib/python3.6/site-packages/numpy/core/records.py:507: FutureWarning: Numpy has detected that you may be viewing or writing to an array returned by selecting multiple fields in a structured array. 

This code may break in numpy 1.13 because this will return a view instead of a copy -- see release notes for details.
  return obj.view(dtype=(self.dtype.type, obj.dtype))


In [40]:
m.run_model()

FloPy is using the following executable to run the model: /Users/jeremyw/bin/mfnwt

                                  MODFLOW-NWT-SWR1 
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUNDWATER-FLOW MODEL
                             WITH NEWTON FORMULATION
                             Version 1.1.3, 8/01/2017                        
                    BASED ON MODFLOW-2005 Version 1.11.0 08/08/2013                       

                    SWR1 Version 1.04.0 09/15/2016                       

 Using NAME file: freyberg.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2018/01/02 21:31:50

 Solving:  Stress period:     1    Time step:     1    Groundwater-Flow Eqn.
 Solving:  Stress period:     2    Time step:     1    Groundwater-Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2018/01/02 21:31:50
 Elapsed run time:  0.081 Seconds

  Normal termination of simulation


(True, [])